In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
clean_data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/"
output_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/"

In [ ]:
pop = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/popolazione_piemonte/population_2002_2019_per_age.csv.csv')
pop.drop(columns=pop.columns[:2], inplace=True)
pop.head()

In [ ]:
popolazione = dict()
for a in list(range(2002, 2020, 1)):
    popolazione[a] = pop.loc[:, [str(a), 'Eta', 'Comune']].pivot_table(values=str(a), columns='Eta', index='Comune').reset_index()

In [ ]:
confini = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/confini_piemonte/confini_piemonte.shp')
confini.head()

In [ ]:
POPOLAZIONE = dict.fromkeys(popolazione.keys())
for a in POPOLAZIONE.keys():
    POPOLAZIONE[a] = popolazione[a].merge(confini[['Codice Com', 'geometry']].rename(columns={'Codice Com': 'Comune'}), on='Comune', how='inner').drop(columns='Comune')

In [ ]:
popolazione[2002].loc[:, popolazione[2002].columns[1:6]].sum(axis=1)

In [ ]:
for a, df in POPOLAZIONE.items():
    cols = [str(c) for c in list(range(0, 9))]
    df['meno di 9 anni'] = df.loc[:, cols].sum(axis=1)
    df.drop(columns=cols, inplace=True)

    cols = [str(c) for c in list(range(9, 15))]
    df['9-14 anni'] = df.loc[:, cols].sum(axis=1)
    df.drop(columns=cols, inplace=True)

    cols = [str(c) for c in list(range(15, 25))]
    df['15-24 anni'] = df.loc[:, cols].sum(axis=1)
    df.drop(columns=cols, inplace=True)

    cols = [str(c) for c in list(range(25, 50))]
    df['25-49 anni'] = df.loc[:, cols].sum(axis=1)
    df.drop(columns=cols, inplace=True)

    cols = [str(c) for c in list(range(50, 65))]
    df['50-64 anni'] = df.loc[:, cols].sum(axis=1)
    df.drop(columns=cols, inplace=True)

    cols = [str(c) for c in list(range(65, 100))] + ['100 e oltre']
    df['piu di 65 anni'] = df.loc[:, cols].sum(axis=1)
    df.drop(columns=cols, inplace=True)

In [ ]:
for a, df in POPOLAZIONE.items():
    save_clean_data(gpd.GeoDataFrame(POPOLAZIONE[a]), str(a), output_folder+'per_comune/popolazione')

---